In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import time

! pip install countryinfo
from countryinfo import CountryInfo

In [2]:
df = pd.read_csv("df0_5000.csv")
df.shape

/tmp/ipykernel_11982/3813456554.py:1: DtypeWarning: Columns (17,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("df0_5000.csv")


(4988, 2215)

In [3]:
def listify_geo_markets(df):
    l = []
    for i,val in enumerate(df['geographical markets']):
        if val is not np.nan:
            l.append(val.split(', '))
        else :
            l.append(np.nan)
    return l

In [4]:
def get_location_list(df: pd.DataFrame)-> list : 
    countries = []
    for list in df['geographical markets']:
        if list is not np.nan:
            for country in list:
                if country not in countries:
                    countries.append(country.lower())
    
    return countries

In [6]:
new_df = df.copy()

In [8]:
new_df['geographical markets'] = listify_geo_markets(df)
new_df['geographical markets']

0       [australia, canada, france, india, united king...
1                                                     NaN
2                  [canada, mexico, spain, united states]
3                                 [global, united states]
4       [north america, europe, global, france, german...
                              ...                        
4983                                                  NaN
4984                                             [global]
4985                                                  NaN
4986                                      [north america]
4987                                                  NaN
Name: geographical markets, Length: 4988, dtype: object

In [11]:
countries = get_location_list(new_df)
len(countries)

146

In [57]:
def get_api_data(countries):
    api = "https://restcountries.com/v3.1/"
    api_v = ['name/','region/','subregion/']

    d = {}
    for country in countries:
        time.sleep(0.5)
        for v in api_v:
            r = requests.get(f"{api}{v}{country}")
            if r.status_code == 200:
                d.update({country : r.json()[0]['population']})
                break
    
    return d


In [91]:
# pop_dict

{'afghanistan': 40218234,
 'africa': 59308690,
 'albania': 2837743,
 'americas': 25987,
 'angola': 32866268,
 'argentina': 45376763,
 'armenia': 2963234,
 'asia': 21919000,
 'australia': 25687041,
 'austria': 8917205,
 'azerbaijan': 10110116,
 'bahrain': 1701583,
 'bangladesh': 164689383,
 'belarus': 9398861,
 'belgium': 11555997,
 'bolivia': 11673029,
 'bosnia and herzegovina': 3280815,
 'brazil': 212559409,
 'bulgaria': 6927288,
 'cambodia': 16718971,
 'cameroon': 26545864,
 'canada': 38005238,
 'central america': 9904608,
 'central asia': 6031187,
 'chad': 16425859,
 'chile': 19116209,
 'china': 1402112000,
 'colombia': 50882884,
 'congo': 89561404,
 'costa rica': 5094114,
 'cuba': 11326616,
 'cyprus': 1207361,
 'czech republic': 10698896,
 'denmark': 5831404,
 'dominican republic': 71991,
 'eastern africa': 114963583,
 'eastern europe': 9398861,
 'ecuador': 17643060,
 'egypt': 102334403,
 'el salvador': 6486201,
 'estonia': 1331057,
 'ethiopia': 114963583,
 'europe': 9398861,
 'fin

In [33]:
def get_driver():
    from selenium import webdriver
    # from selenium.webdriver.firefox.service import Service
    # from webdriver_manager.firefox import GeckoDriverManager
    

    options = webdriver.ChromeOptions()
    
    # options.add_argument("disable-blink-features=AutomationControlled")
    

    driver = webdriver.Chrome("../chromedriver", options = options)
    return driver

In [13]:
def get_world_pop() -> int :
    """Get world population by scraping www.worldometers.info"""
    from selenium.webdriver.common.by import By
    
    driver = get_driver()

    url = "https://www.worldometers.info/world-population/"
    driver.get(url)
    population = driver.find_element(By.CLASS_NAME, "maincounter-number").text
    driver.quit()
    pop = population.split(',')
    num = ''.join(pop)
    return int(num)



In [48]:
def get_countries_pop(countries_list) -> dict:
    """Get countries population by scraping www.worldometers.info"""
    from selenium.webdriver.common.by import By
    
    driver = get_driver()
    url = "https://www.worldometers.info/world-population/population-by-country/"
    driver.get(url)
    d = {}
    table = driver.find_element(By.TAG_NAME, "tbody").find_elements(By.TAG_NAME, "tr")

    for row in table:
        elements = row.find_elements(By.TAG_NAME, "td")
        d.update({elements[1].text.lower(): elements[2].text.lower()})
    
    driver.quit()

    return d
    

In [49]:
def get_countries_pop_countryinfo(countries_list) -> dict:
    """Get countries population from country info module"""
    api_data = []
    d = {}
    for country in countries_list:
        try:
            d.update({country: CountryInfo(country).population()})
        except:
            # print(f'Error on {country}')
            api_data.append(country)
    
    d.update(get_api_data(api_data))

    return d

In [52]:
country_pop = get_countries_pop(countries)
country_pop

/tmp/ipykernel_11982/1855897883.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("../chromedriver", options = options)


{'china': '1,439,323,776',
 'india': '1,380,004,385',
 'united states': '331,002,651',
 'indonesia': '273,523,615',
 'pakistan': '220,892,340',
 'brazil': '212,559,417',
 'nigeria': '206,139,589',
 'bangladesh': '164,689,383',
 'russia': '145,934,462',
 'mexico': '128,932,753',
 'japan': '126,476,461',
 'ethiopia': '114,963,588',
 'philippines': '109,581,078',
 'egypt': '102,334,404',
 'vietnam': '97,338,579',
 'dr congo': '89,561,403',
 'turkey': '84,339,067',
 'iran': '83,992,949',
 'germany': '83,783,942',
 'thailand': '69,799,978',
 'united kingdom': '67,886,011',
 'france': '65,273,511',
 'italy': '60,461,826',
 'tanzania': '59,734,218',
 'south africa': '59,308,690',
 'myanmar': '54,409,800',
 'kenya': '53,771,296',
 'south korea': '51,269,185',
 'colombia': '50,882,891',
 'spain': '46,754,778',
 'uganda': '45,741,007',
 'argentina': '45,195,774',
 'algeria': '43,851,044',
 'sudan': '43,849,260',
 'ukraine': '43,733,762',
 'iraq': '40,222,493',
 'afghanistan': '38,928,346',
 'pol

In [44]:
sum(country_pop.values())

6802127507

In [54]:
pop = get_world_pop()
pop

/tmp/ipykernel_11982/1855897883.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("../chromedriver", options = options)


7947726879

In [55]:
country_pop['global'] = pop
country_pop['global']

7947726879

In [58]:
# List definistions
not_in_list = []

for country in countries:
    if country not in country_pop.keys():
        not_in_list.append(country)

print(len(country_pop))

country_pop.update(get_api_data(not_in_list))
print(len(country_pop))

236
259


In [61]:
country_pop

{'china': '1,439,323,776',
 'india': '1,380,004,385',
 'united states': '331,002,651',
 'indonesia': '273,523,615',
 'pakistan': '220,892,340',
 'brazil': '212,559,417',
 'nigeria': '206,139,589',
 'bangladesh': '164,689,383',
 'russia': '145,934,462',
 'mexico': '128,932,753',
 'japan': '126,476,461',
 'ethiopia': '114,963,588',
 'philippines': '109,581,078',
 'egypt': '102,334,404',
 'vietnam': '97,338,579',
 'dr congo': '89,561,403',
 'turkey': '84,339,067',
 'iran': '83,992,949',
 'germany': '83,783,942',
 'thailand': '69,799,978',
 'united kingdom': '67,886,011',
 'france': '65,273,511',
 'italy': '60,461,826',
 'tanzania': '59,734,218',
 'south africa': '59,308,690',
 'myanmar': '54,409,800',
 'kenya': '53,771,296',
 'south korea': '51,269,185',
 'colombia': '50,882,891',
 'spain': '46,754,778',
 'uganda': '45,741,007',
 'argentina': '45,195,774',
 'algeria': '43,851,044',
 'sudan': '43,849,260',
 'ukraine': '43,733,762',
 'iraq': '40,222,493',
 'afghanistan': '38,928,346',
 'pol

In [64]:

country_pop_int = {}
for key,val in country_pop.items():
    if type(val) == str:
        popu = val.split(',')
        if len(popu) > 1:
            country_pop_int.update({key : int(''.join(popu))})
    else:
        country_pop_int.update({key : val})
country_pop_int

{'china': 1439323776,
 'india': 1380004385,
 'united states': 331002651,
 'indonesia': 273523615,
 'pakistan': 220892340,
 'brazil': 212559417,
 'nigeria': 206139589,
 'bangladesh': 164689383,
 'russia': 145934462,
 'mexico': 128932753,
 'japan': 126476461,
 'ethiopia': 114963588,
 'philippines': 109581078,
 'egypt': 102334404,
 'vietnam': 97338579,
 'dr congo': 89561403,
 'turkey': 84339067,
 'iran': 83992949,
 'germany': 83783942,
 'thailand': 69799978,
 'united kingdom': 67886011,
 'france': 65273511,
 'italy': 60461826,
 'tanzania': 59734218,
 'south africa': 59308690,
 'myanmar': 54409800,
 'kenya': 53771296,
 'south korea': 51269185,
 'colombia': 50882891,
 'spain': 46754778,
 'uganda': 45741007,
 'argentina': 45195774,
 'algeria': 43851044,
 'sudan': 43849260,
 'ukraine': 43733762,
 'iraq': 40222493,
 'afghanistan': 38928346,
 'poland': 37846611,
 'canada': 37742154,
 'morocco': 36910560,
 'saudi arabia': 34813871,
 'uzbekistan': 33469203,
 'peru': 32971854,
 'angola': 32866272,

In [73]:
pop_per = {}

for key, val in country_pop_int.items():
    pop_per.update({key.lower(): (val/pop)})


# pop_per.update({'unknown': ((1-sum_per)*pop) / val })


In [77]:
[print(i,j) for i,j in pop_per.items()]

china 0.1810987969155149
india 0.1736351042266358
united states 0.041647461725766735
indonesia 0.034415326440409254
pakistan 0.027793146815809196
brazil 0.026744680615741628
nigeria 0.02593692412162217
bangladesh 0.0207215705203903
russia 0.018361786234199557
mexico 0.016222594832828806
japan 0.015913538918176002
ethiopia 0.014464964605636394
philippines 0.013787725681608692
egypt 0.012875933654740277
vietnam 0.012247348264721365
dr congo 0.01126880734120909
turkey 0.0106117218525521
iran 0.01056817254527601
germany 0.01054187483736757
thailand 0.008782382568332844
united kingdom 0.008541563145479096
france 0.008212852806060801
italy 0.007607436304807625
tanzania 0.007515887109537399
south africa 0.007462346266164389
myanmar 0.006845957445236965
kenya 0.006765619505883879
south korea 0.006450798546621773
colombia 0.006402194209069524
spain 0.005882786199351982
uganda 0.005755231363178805
argentina 0.005686628980598114
algeria 0.005517432174961381
sudan 0.005517207708264531
ukraine 0.00

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,